In [221]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

In [96]:
data = pd.read_csv("data.csv");

/Users/daraakdag/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,8,9,10,11,13,14,15,16,17,18,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
descriptors = pd.read_csv("descriptors.csv");

# Import smiles data from txt file

The simplified molecular-input line-entry system (SMILES) is a specification in form of a line notation for describing the structure of chemical species using short ASCII strings. SMILES strings can be imported by most molecule editors for conversion back into two-dimensional drawings or three-dimensional models of the molecules. The smiles data file shows the molecule ids and their respective structure. It was downloaded here:
https://pubchem.ncbi.nlm.nih.gov/bioassay/624202#section=Top
Click on the "Download" tab -> select tested substances -> click "Download service" -> Chose "Smiles" as format and download. 

In [105]:
smilesData = pd.read_csv("smilesData.txt",sep="\t", header=None);

In [112]:
smilesData.head()

,0,1
0,124899260,C1=CC=C(C=C1)CCCCOC2=CC=C(C=C2)CC(=O)NO
1,124899259,C1=CC=NC(=C1)COC2=CC=C(C=C2)CC(=O)NO
2,124899258,C1=CC=C(C=C1)CCCOC2=CC=C(C=C2)CC(=O)NO
3,124899257,CC(C)COC1=CC=C(C=C1)CC(=O)NO
4,124899256,CCCCCCCOC1=CC=C(C=C1)CC(=O)NO


In [119]:
m = Chem.MolFromSmiles(smilesData[1][0])

In [246]:
descList = [i[0] for i in Descriptors._descList]

In [181]:
Descriptors.MaxAbsPartialCharge(m)

0.4936012745869841

In [136]:
from rdkit.Chem import rdMolDescriptors
mList = rdMolDescriptors.CalcAUTOCORR2D(m)

In [161]:
from rdkit.ML.Descriptors import Descriptors 
from rdkit.ML.Descriptors import MoleculeDescriptors

In [164]:
descriptorCalc = Descriptors

# Test

In [183]:
descriptors.head()

,Unnamed: 0,molSIDs,XLogP,MW,LipinskiFailures,nRotB,MLogP,nAtomLAC,nAtomP,nAtomLC,...,RPCG,RNCG,RPCS,RNCS,THSA,TPSA,RHSA,RPSA,bpol,apol
0,1,124897530,1.135,369.168856,0,10,2.89,3,8,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.677761,57.044239
1,2,124897303,4.498,249.095378,0,2,3.11,2,18,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.641484,39.578516
2,3,124753561,4.554,657.230885,0,11,4.21,0,9,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.764452,98.933548
3,4,124753354,8.321,501.146461,1,9,2.89,4,16,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.743589,70.138411
4,5,121286038,4.515,637.224657,0,10,4.32,2,24,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.306245,97.709755


In [54]:
data = data.loc[5:,['PUBCHEM_SID','PUBCHEM_ACTIVITY_SCORE']];

In [71]:
inactives= data.loc[data.PUBCHEM_ACTIVITY_SCORE==0]
actives= data.loc[data.PUBCHEM_ACTIVITY_SCORE!=0]
print("Number of inactives is ",len(inactives))
print("Number of actives is ", len(actives))

Number of inactives is  364035
Number of actives is  13515


In [80]:
selectedInactives = inactives.sample(350)
selectedActives = actives.sample(700)

In [81]:
smallDataset = pd.concat([selectedInactives,selectedActives])

In [88]:
print("Shape of small dataset is",smallDataset.shape)

Shape of small dataset is (1050, 2)


In [92]:
smallDataset.head()

,PUBCHEM_SID,PUBCHEM_ACTIVITY_SCORE
231584,47197529.0,0.0
375920,124879998.0,0.0
358318,99360986.0,0.0
204552,26657698.0,0.0
119006,22403948.0,0.0
